In [4]:
import pandas as pd

file_url = 'https://raw.githubusercontent.com/ajayt0m/Forecasting_Crop_Prices_Maharashtra/main/onion_daily_cleaned.csv'

onion_df = pd.read_csv(file_url)

In [5]:
onion_df

,MIN,MAX,MODAL,Arrival
0,865.145833,2706.875000,2060.604167,11575.00
1,1105.066667,2720.155556,2181.844444,15560.00
2,1129.411765,2741.470588,2141.176471,3954.00
3,967.306122,2754.163265,2167.551020,21441.00
4,1188.836735,2793.714286,2275.469388,21526.00
...,...,...,...,...
999,948.090909,2731.454545,1978.393939,17023.00
1000,1317.647059,2629.470588,2104.411765,10241.00
1001,878.925926,2629.555556,1874.296296,10973.12
1002,1006.000000,2626.200000,1982.000000,18822.00
